In [1]:
import os
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession
import warnings

warnings.filterwarnings("ignore")

# Encontrar o caminho absoluto do diretório atual
notebook_cwd = os.getcwd()

# Definir o caminho correto para a raiz do projeto Kedro
project_path = r"c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project"

# Verificar o diretório atual e o caminho do projeto
print(f"Notebook current working directory: {notebook_cwd}")
print(f"Project path: {project_path}")

# Alterar para o diretório raiz do projeto Kedro
os.chdir(project_path)

# Bootstrap o projeto Kedro
bootstrap_project(project_path)

# Inicialize o contexto do Kedro
with KedroSession.create() as session:
    context = session.load_context()

# Recarregar o catálogo
catalog = context.catalog

# Acessar os parâmetros
params = context.params

# Listar o catálogo
catalog.list()

[09/15/24 02:09:39] INFO     Using                                                                  ]8;id=330942;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=502630;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py#249\249]8;;\
                             'c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framew                
                             ork\project\rich_logging.yml' as logging configuration.                               

Notebook current working directory: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\notebooks\gmferratti\pipeline
Project path: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project


[09/15/24 02:09:41] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=930893;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=271194;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         


[
    'decks_json_partitioned',
    'decks_txt_partitioned',
    'sampled_decks',
    'players',
    'players_with_decks',
    'matches_dataframe',
    'parameters',
    'params:global',
    'params:global.run_date',
    'params:global.run_date.day',
    'params:global.run_date.month',
    'params:global.run_date.year',
    'params:global.user',
    'params:global.user.project_path',
    'params:preprocessing',
    'params:preprocessing.webscraper',
    'params:preprocessing.webscraper.zip_url',
    'params:preprocessing.webscraper.zip_folder',
    'params:preprocessing.webscraper.log_folder',
    'params:preprocessing.webscraper.deck_cards',
    'params:preprocessing.webscraper.sample_size_ratio',
    'params:simulation',
    'params:simulation.n_players',
    'params:simulation.matches_per_player',
    'params:simulation.hand_size_stop',
    'params:simulation.max_turns',
    'params:simulation.max_mulligans',
    'params:simulation.mulligan_prob',
    'params:simulation.extra_land_

In [18]:
from typing import List, Dict
from classes.deck import Deck
from classes.player import Player
from classes.player_tracker import PlayerTracker
from src.mtg_project.pipelines.utils import setup_logger
import pandas as pd
import numpy as np
from faker import Faker
import random

In [3]:
def create_players(n_players: int):
    """
    Cria uma lista de objetos Player com nomes aleatórios.

    Args:
        n_players (int): Número de jogadores a serem criados.

    Returns:
        List[Player]: Lista de objetos Player com nomes gerados aleatoriamente.
    """
    # Inicializando o gerador de dados falsos Faker
    fake = Faker()
    
    # Gerando uma lista de nomes aleatórios usando o Faker
    player_names = [fake.first_name() + " " + fake.last_name() for _ in range(n_players)]
    
    # Criando uma lista de objetos Player a partir dos nomes gerados
    players = [Player(name) for name in player_names]

    # Retornando a lista de objetos Player
    return players

n_players = catalog.load("params:simulation.n_players")
players = create_players(n_players)
catalog.save("players", players)

[09/15/24 02:09:43] INFO     Loading data from params:simulation.n_players (MemoryDataset)...   ]8;id=730749;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=802885;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Saving data to players (PickleDataset)...                          ]8;id=908595;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=810786;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [4]:
import random
import os
from typing import List, Dict

def assign_decks_to_players(
        players: List[Player], 
        sampled_decks: Dict[str, str],
        log_folder: str) -> List[Player]:
    """
    Função para atribuir decks aleatórios a cada player na lista de players.

    A função tentará atribuir um deck a cada player chamando o método assign_deck().
    Caso ocorra algum erro na atribuição, tentará com outro deck disponível.

    Args:
        players (list): Lista de objetos Player.
        sampled_decks (dict): Dicionário com os nomes e caminhos dos decks.
        log_folder (str): Caminho da pasta para salvar o log.

    Returns:
        List[Player]: Lista de objetos Player com decks atribuídos.
    """
    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'decks_assignment.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Configura o logger geral
    logger = setup_logger("validate_decks", log_filepath)
    
    # Log de início da validação
    logger.info("Validating decks...")

    # Convertemos as chaves do dicionário para uma lista de nomes de decks disponíveis
    available_decks = list(sampled_decks.keys())
    
    for player in players:
        assigned = False
        while not assigned and available_decks:
            try:
                # Seleciona um deck aleatório da lista de decks disponíveis
                deck_name = random.choice(available_decks)

                # Obter o caminho completo do deck a partir do dicionário sampled_decks
                deck_path = sampled_decks[deck_name]

                # Cria um novo objeto Deck
                deck = Deck()

                # Carrega o deck a partir do arquivo .txt no caminho obtido
                deck.load_deck_from_txt(deck_path)

                # Atribui o deck ao player
                player.assign_deck(deck)
                logger.info(f"Deck '{deck_name}' assigned to player '{player.name}'")
                
                # Remove o deck da lista de decks disponíveis para evitar reutilização
                available_decks.remove(deck_name)

                assigned = True  # Deck atribuído com sucesso
            except Exception as e:
                # Em caso de erro, tenta outro deck
                logger.error(f"Failed to assign deck '{deck_name}' to player '{player.name}': {e}")
                continue
        
        
        # Se não houver mais decks disponíveis e não conseguir atribuir, lança um erro
        if not assigned:
            raise ValueError(f"No available decks left to assign to player '{player.name}'.")

    logger.info("Deck assignment process completed.")

    return players

players = catalog.load("players")
sampled_decks = catalog.load("sampled_decks")
log_folder = catalog.load("params:simulation.log_folder")
players_with_decks = assign_decks_to_players(players, sampled_decks, log_folder)
catalog.save("players_with_decks", players_with_decks)

                    INFO     Loading data from players (PickleDataset)...                       ]8;id=838290;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=47215;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from sampled_decks (PickleDataset)...                 ]8;id=109394;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=519166;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from params:simulation.log_folder (MemoryDataset)...  ]8;id=630448;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=742124;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

[09/15/24 02:10:28] INFO     Saving data to players_with_decks (PickleDataset)...               ]8;id=314954;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=26114;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [5]:
def simulate_player_matches(params: dict, players_with_decks: list) -> pd.DataFrame:
    """
    Simulates Magic: The Gathering matches for a list of players based on the provided simulation parameters.

    Parameters:
    -----------
    params : dict
        A dictionary containing the simulation parameters, including:
        - 'max_mulligans': Maximum number of mulligans allowed per player.
        - 'mulligan_prob': Probability of a player choosing to mulligan.
        - 'hand_size_stop': Minimum hand size at which the simulation will stop.
        - 'max_turns': Maximum number of turns per match.
        - 'extra_land_prob': Probability of playing an extra land during a turn.
        - 'matches_per_player': Number of matches to simulate per player.
        - 'log_folder': Folder path for logging the simulation process.
    
    players_with_decks : list
        A list of Player objects, each with an assigned deck to be used in the simulation.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the match data for all players across all matches and turns, including:
        - Player attributes at each turn.
        - Match number for each simulation.
    """
    
    # Atribuir os parâmetros
    max_mulligans = params["max_mulligans"]
    mulligan_prob = params["mulligan_prob"]
    hand_size_stop = params["hand_size_stop"]
    max_turns = params["max_turns"]
    extra_land_prob = params["extra_land_prob"]
    matches_per_player = params["matches_per_player"]
    log_folder = params["log_folder"]

    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'player_matches.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Configura o logger geral
    logger = setup_logger("player_matches", log_filepath)

    # Log de início da validação
    logger.info("Initiating simulations...")

    # Inicializa o tracker para armazenar os dados
    tracker = PlayerTracker()

    # Loop através dos jogadores e realizar as simulações de partidas
    for player in players_with_decks:
        for match in range(matches_per_player):
            # Simular várias partidas para o jogador
            player.play_a_match(tracker, 
                                max_mulligans, 
                                mulligan_prob, 
                                max_turns, 
                                hand_size_stop, 
                                extra_land_prob)

    # Obter os dados de todas as partidas e turnos
    matches_dataframe = tracker.get_data()
    
    return matches_dataframe

# Chamada da função
params = catalog.load("params:simulation")
players_with_decks = catalog.load("players_with_decks")
matches_df = simulate_player_matches(params, players_with_decks)
catalog.save("matches_dataframe",matches_df)

                    INFO     Loading data from params:simulation (MemoryDataset)...             ]8;id=278107;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=190474;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from players_with_decks (PickleDataset)...            ]8;id=62743;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=969381;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    WARNING  Hunter Lopez couldn't play any spells.                                   ]8;id=604122;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=23698;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Hunter Lopez couldn't play any spells.                                   ]8;id=333;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=802276;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Hunter Lopez couldn't play any spells.                                   ]8;id=818643;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=700820;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

...

                    WARNING  Chelsea Bowen couldn't play any spells.                                  ]8;id=305000;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=782867;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Chelsea Bowen couldn't play any spells.                                  ]8;id=357463;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=495029;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Chelsea Bowen couldn't play any spells.                                  ]8;id=767532;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=723431;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    WARNING  Chelsea Bowen couldn't play any spells.                                  ]8;id=14039;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=259814;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#371\371]8;;\

                    INFO     Saving data to matches_dataframe (ParquetDataset)...               ]8;id=214635;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=559459;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [14]:
matches_df

,name,deck_name,deck_colors,match,turn,mulligan_count,lands_played,spells_played,mana_pool,spent_mana,hand_size,library_size,graveyard_size,full_hand,full_graveyard,cum_spent_mana,cum_spells_played,spell_ratio,land_ratio
0,Hunter Lopez,sorin,"[B, W]",1,0,0,0,0,0,0,7,53,0,"Hand(7 cards: Swamp, Swamp, Swamp, Plains, Swa...",Graveyard(0 cards: ),0,0,0.000000,0.000000
1,Hunter Lopez,sorin,"[B, W]",1,1,0,1,0,1,0,7,52,0,"Hand(7 cards: Swamp, Swamp, Plains, Swamp, Unm...",Graveyard(0 cards: ),0,0,0.000000,0.500000
2,Hunter Lopez,sorin,"[B, W]",1,2,0,2,1,3,2,5,51,1,"Hand(5 cards: Plains, Swamp, Unmake, Field of ...",Graveyard(1 cards: Sorin's Thirst),2,1,0.333333,0.666667
3,Hunter Lopez,sorin,"[B, W]",1,3,0,2,2,5,4,3,50,2,"Hand(3 cards: Unmake, Mesmeric Fiend, Bloodrag...","Graveyard(2 cards: Sorin's Thirst, Field of So...",6,3,0.500000,0.500000
4,Hunter Lopez,sorin,"[B, W]",1,4,0,1,4,6,6,1,49,4,Hand(1 cards: Mesmeric Fiend),"Graveyard(4 cards: Sorin's Thirst, Field of So...",12,7,0.800000,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Chelsea Bowen,liliana death wielder,"[G, B]",10,3,0,2,2,4,3,4,50,2,"Hand(4 cards: Swamp, Karplusan Forest, Karplus...","Graveyard(2 cards: Tattered Mummy, Baleful Ammit)",1120,1583,0.500000,0.500000
393,Chelsea Bowen,liliana death wielder,"[G, B]",10,4,0,1,3,5,4,3,49,3,"Hand(3 cards: Karplusan Forest, Karplusan Fore...","Graveyard(3 cards: Tattered Mummy, Baleful Amm...",1124,1586,0.600000,0.200000
394,Chelsea Bowen,liliana death wielder,"[G, B]",10,5,0,1,4,6,6,2,48,4,"Hand(2 cards: Karplusan Forest, Foul Orchard)","Graveyard(4 cards: Tattered Mummy, Baleful Amm...",1130,1590,0.666667,0.166667
395,Chelsea Bowen,liliana death wielder,"[G, B]",10,6,0,1,5,7,1,1,47,5,Hand(1 cards: Foul Orchard),"Graveyard(5 cards: Tattered Mummy, Baleful Amm...",1131,1595,0.714286,0.142857


In [38]:
# Carregar o dataframe
matches_df = catalog.load("matches_dataframe")
matches_df["spent_mana"] = matches_df["spent_mana"].astype(int)

# Creating cumulative variables
matches_df['cum_mana_pool'] = matches_df['mana_pool'].cumsum()
matches_df["cum_spent_mana"] = matches_df["spent_mana"].cumsum()
matches_df["cum_spells_played"] = matches_df["spells_played"].cumsum()

# Creating ratio variables
matches_df['spell_ratio'] = (matches_df['spells_played'] / (matches_df['turn'] + 1)).round(2)
matches_df['land_ratio'] = (matches_df['lands_played'] / (matches_df['turn'] + 1)).round(2)

# Creating target variable
matches_df['mana_curve_efficiency'] = matches_df['cum_spent_mana'] / matches_df['cum_mana_pool']
matches_df['mana_curve_efficiency'].replace([float('inf'), -float('inf')], 0, inplace=True)
matches_df['mana_curve_efficiency'].fillna(0, inplace=True)
matches_df['mana_curve_efficiency'] = matches_df['mana_curve_efficiency'].round(2)

# Expand deck_colors into multiple columns (One-Hot Encoding for individual colors)
all_colors = ['W', 'U', 'B', 'R', 'G']

# Criar uma coluna para cada cor
for color in all_colors:
    matches_df[f'{color}'] = (matches_df['deck_colors'].apply(lambda x: 1 if color in x else 0)).astype("category")

# Drop the original deck_colors column
matches_df.drop(columns=['deck_colors'], inplace=True)

# Visualizando as primeiras linhas para confirmar as alterações
matches_df.head()

[09/15/24 02:49:49] INFO     Loading data from matches_dataframe (ParquetDataset)...            ]8;id=825895;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=451946;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

,name,deck_name,match,turn,mulligan_count,lands_played,spells_played,mana_pool,spent_mana,hand_size,...,cum_spent_mana,cum_spells_played,spell_ratio,land_ratio,mana_curve_efficiency,W,U,B,R,G
0,Hunter Lopez,sorin,1,0,0,0,0,0,0,7,...,0,0,0.00,0.00,0.00,1,0,1,0,0
1,Hunter Lopez,sorin,1,1,0,1,0,1,0,7,...,0,0,0.00,0.50,0.00,1,0,1,0,0
2,Hunter Lopez,sorin,1,2,0,2,1,3,2,5,...,2,1,0.33,0.67,0.50,1,0,1,0,0
3,Hunter Lopez,sorin,1,3,0,2,2,5,4,3,...,6,3,0.50,0.50,0.67,1,0,1,0,0
4,Hunter Lopez,sorin,1,4,0,1,4,6,6,1,...,12,7,0.80,0.20,0.80,1,0,1,0,0


In [33]:
threshold = 0.85

# Remover colunas do tipo 'object' ou 'category'
matches_df_numeric = matches_df.select_dtypes(include=[np.number])

# Calcular a matriz de correlação
corr_matrix = matches_df_numeric.corr().abs()

# Criar uma máscara para identificar as correlações acima do limiar (excluindo a diagonal da matriz)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Identificar colunas com alta correlação (acima do limiar)
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Remover as colunas altamente correlacionadas
matches_df_cleaned = matches_df.drop(columns=to_drop)

# Exibindo as colunas removidas e o dataframe resultante
print(f"Colunas removidas por alta correlação: {to_drop}")
matches_df_cleaned.head()

Colunas removidas por alta correlação: ['spells_played', 'hand_size', 'library_size', 'graveyard_size', 'cum_spells_played']


,name,deck_name,deck_colors,match,turn,mulligan_count,lands_played,mana_pool,spent_mana,full_hand,full_graveyard,cum_spent_mana,spell_ratio,land_ratio,color_W,color_U,color_B,color_R,color_G
0,Hunter Lopez,sorin,"[B, W]",1,0,0,0,0,0,"Hand(7 cards: Swamp, Swamp, Swamp, Plains, Swa...",Graveyard(0 cards: ),0,0.000000,0.000000,1,0,1,0,0
1,Hunter Lopez,sorin,"[B, W]",1,1,0,1,1,0,"Hand(7 cards: Swamp, Swamp, Plains, Swamp, Unm...",Graveyard(0 cards: ),0,0.000000,0.500000,1,0,1,0,0
2,Hunter Lopez,sorin,"[B, W]",1,2,0,2,3,2,"Hand(5 cards: Plains, Swamp, Unmake, Field of ...",Graveyard(1 cards: Sorin's Thirst),2,0.333333,0.666667,1,0,1,0,0
3,Hunter Lopez,sorin,"[B, W]",1,3,0,2,5,4,"Hand(3 cards: Unmake, Mesmeric Fiend, Bloodrag...","Graveyard(2 cards: Sorin's Thirst, Field of So...",6,0.500000,0.500000,1,0,1,0,0
4,Hunter Lopez,sorin,"[B, W]",1,4,0,1,6,6,Hand(1 cards: Mesmeric Fiend),"Graveyard(4 cards: Sorin's Thirst, Field of So...",12,0.800000,0.200000,1,0,1,0,0
